<a href="https://colab.research.google.com/github/malewi/Terrain-Identification-for-Time-Series-Data/blob/master/projC1_Updated_(4_17).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Flatten, Dense, Reshape, Dropout, UpSampling1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from scipy import interpolate
import tensorflow as tf
import fnmatch
from scipy import stats
from imblearn.over_sampling import SMOTE


# Set up the file paths for the dataset
xFiles = []
yFiles = []
xTimeFiles = []
yTimeFiles = []

for file in os.listdir("/content/TrainingData"):
    if fnmatch.fnmatch(file, "*__x.csv"):
        xFiles.append(file)
    elif fnmatch.fnmatch(file, "*__y.csv"):
        yFiles.append(file)
    elif fnmatch.fnmatch(file, "*__x_time.csv"):
        xTimeFiles.append(file)
    elif fnmatch.fnmatch(file, "*__y_time.csv"):
        yTimeFiles.append(file)

subjects = list(set([x[:9] for x in xFiles]))
files = sorted(set([file[:14] for file in xFiles]))

# Create empty dataframes for x and y data
xData = pd.DataFrame()
yData = pd.DataFrame()

print(yTimeFiles)

['subject_006_03__y_time.csv', 'subject_004_01__y_time.csv', 'subject_001_06__y_time.csv', 'subject_004_02__y_time.csv', 'subject_002_02__y_time.csv', 'subject_002_05__y_time.csv', 'subject_007_04__y_time.csv', 'subject_007_03__y_time.csv', 'subject_005_02__y_time.csv', 'subject_007_01__y_time.csv', 'subject_006_01__y_time.csv', 'subject_001_05__y_time.csv', 'subject_001_02__y_time.csv', 'subject_001_04__y_time.csv', 'subject_005_01__y_time.csv', 'subject_007_02__y_time.csv', 'subject_003_03__y_time.csv', 'subject_002_03__y_time.csv', 'subject_001_03__y_time.csv', 'subject_001_07__y_time.csv', 'subject_001_08__y_time.csv', 'subject_005_03__y_time.csv', 'subject_003_02__y_time.csv', 'subject_003_01__y_time.csv', 'subject_002_04__y_time.csv', 'subject_006_02__y_time.csv', 'subject_008_01__y_time.csv', 'subject_002_01__y_time.csv']


In [ ]:
cd '/content/TrainingData'

/content/TrainingData


In [ ]:
# Loop through each subject and read in the corresponding x, y, x_time, and y_time files
for file in files:
    y_data = pd.read_csv(file + '__y.csv', names=['label'])
    y_time = pd.read_csv(file + '__y_time.csv', names=['y_time'])
    x_data = pd.read_csv(file + '__x.csv', names=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z'])
    x_time = pd.read_csv(file + '__x_time.csv', names=['x_time'])
    
    # Add time column to x_data and y_data dataframes
    x_data['time'] = x_time['x_time']
    y_data['time'] = y_time['y_time']
    
    # Merge x_data and y_data on the time column
    data = pd.merge(x_data, y_data, how='outer', on='time')
    data.sort_values("time", inplace=True)
    
    # Forward-fill missing label values
    data['label'].fillna(method="bfill", inplace=True)

    # Interpolate missing values in x_data
    data = data.interpolate()
    
    # Add subject ID column
    data['subject'] = file

    new_data_list = []

    i = .2
    for datapoint in data.values:
      if math.isclose(datapoint[6], i/10):
        new_data_list.append(datapoint)
        i += 1
    
    new_data = pd.DataFrame(new_data_list, columns=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'time', 'label', 'subject'])

    # Append data to xData and yData dataframes
    xData = pd.concat([xData, new_data], ignore_index=True)
    yData = pd.concat([yData, new_data[['label', 'subject', 'time']]], ignore_index=True)

    if fnmatch.fnmatch(file, "subject_001_01"):
      # Set the index of xData and yData dataframes
      x_01Data = xData
      y_01Data = yData
      x_01Data.set_index(['subject', 'time'], inplace=True)
      y_01Data.set_index(['subject', 'time'], inplace=True)

      y_01 = xData.label
      X_01 = xData.drop('label', axis=1)

# Set the index of xData and yData dataframes
xData.set_index(['subject', 'time'], inplace=True)
yData.set_index(['subject', 'time'], inplace=True)

y = xData.label
X = xData.drop('label', axis=1)

oversample = SMOTE(sampling_strategy={3.0: 255000, 1.0: 245600, 2.0: 245600})
Xsmote, Ysmote = oversample.fit_resample(X, y)

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
Xsmote = scaler.fit_transform(Xsmote)
Xsmote = pd.DataFrame(Xsmote)

acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z
subject time                                                            
NaN     NaN   1.976206  9.556683  1.752851  0.003280  0.048418  0.099815
        NaN   1.542927  9.476271  1.710742  0.002312 -0.000703  0.016000
        NaN   1.060052  9.466343  1.705366  0.017568  0.046097  0.114381
        NaN   1.880591  9.491277  1.707977  0.011733  0.063126  0.148809
        NaN   2.157620  9.496302  1.752066  0.002755  0.003292  0.024537
        NaN   1.550724  9.493311  1.695947  0.004958 -0.015990 -0.008234
        NaN   1.715569  9.443409  1.695297  0.002838  0.005054  0.025852
        NaN   1.894559  9.520336  1.734895  0.002224  0.018031  0.020313
        NaN   2.095658  9.544117  1.726282  0.010474  0.022432 -0.010853
        NaN   1.790372  9.521457  1.702098 -0.002420 -0.051552 -0.089322
        NaN   1.549402  9.451460  1.695287 -0.002817 -0.024077 -0.050276
        NaN   1.814001  9.459890  1.729296  0.007444 -0.006398 -0.002388
        NaN   1.777070  9.402418  1.703399  0.000857 -0.027673 -0.043773
        NaN   1.771814  9.472504  1.678188  0.001470 -0.022357 -0.029663
        NaN   1.788206  9.572613  1.749178 -0.003205 -0.059942 -0.073048
        NaN   1.686841  9.526294  1.682835 -0.012268 -0.076704 -0.099141
        NaN   1.435216  9.477539  1.649674 -0.012127 -0.047551 -0.106491
        NaN   1.245775  9.608495  1.695778 -0.010158 -0.062211 -0.111410
        NaN   1.233833  9.570998  1.748069 -0.008232 -0.051445 -0.061833
        NaN   1.435549  9.493458  1.698726 -0.003527 -0.020920  0.026785

In [ ]:
Ysmote = to_categorical(Ysmote, 4)
y_frame = pd.DataFrame(Ysmote)

In [ ]:
window = 10
window_x = []
window_y = []
for i in range(0, len(Xsmote) - window, window + 1):
  window_x.append([Xsmote.values[ i : i + window ]])
  window_y.append([y_frame.values[ i : i + window ]])
window_x = np.array(window_x).reshape( -1, window, 6 )
window_y = np.array(window_y).reshape( -1, window, 4 )

(58656, 20, 4)
(58656, 20, 6)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(window_x, window_y, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = Sequential()
model.add(Conv1D(64, 8, activation='relu', padding='same', input_shape=(20, 6)))
model.add(Conv1D(64, 8, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 8, activation='relu', padding='same'))
model.add(Conv1D(128, 8, activation='relu', padding='same'))
model.add(UpSampling1D(2))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))
model.add(Conv1D(256, 8, activation='relu', padding='same'))
model.add(Conv1D(256, 8, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(UpSampling1D(2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.0001), metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=.2, epochs=25, batch_size=32)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 20, 64)            3136      
                                                                 
 conv1d_1 (Conv1D)           (None, 20, 64)            32832     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 10, 128)           65664     
                                                                 
 conv1d_3 (Conv1D)           (None, 10, 128)           131200    
                                                                 
 up_sampling1d (UpSampling1D  (None, 20, 128)          0         
 )                                                      

In [ ]:
from sklearn.metrics import classification_report
from collections import Counter

y_pred = model.predict(X_test)
y_hat=[]
for i in y_pred:
  for j in i:
    y_hat.append(np.argmax(j))
y_test_flat=[]
for i in y_test:
  for j in i:
    y_test_flat.append(np.argmax(j))

print(classification_report(y_test_flat, y_hat))

367/367 [==============================] - 8s 22ms/step
Counter({0: 96040, 1: 48090, 2: 46792, 3: 43718})
[0 1 2 3]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     92969
           1       0.92      0.93      0.93     47564
           2       0.98      0.99      0.99     46416
           3       0.91      0.84      0.87     47691

    accuracy                           0.94    234640
   macro avg       0.94      0.93      0.94    234640
weighted avg       0.94      0.94      0.94    234640



In [ ]:
xFiles = []

xTimeFiles = []
yTimeFiles = []

for file in os.listdir("/content/TestData"):
    if fnmatch.fnmatch(file, "*__x.csv"):
        xFiles.append(file)
    elif fnmatch.fnmatch(file, "*__y.csv"):
        yFiles.append(file)
    elif fnmatch.fnmatch(file, "*__x_time.csv"):
        xTimeFiles.append(file)
    elif fnmatch.fnmatch(file, "*__y_time.csv"):
        yTimeFiles.append(file)

subjects = list(set([x[:7] for x in xFiles]))

['subject_001_01__y_time.csv']


In [ ]:
cd '/content/TestData'

/content/TrueTestData


In [ ]:
# Get the list of unique subject IDs from the file names
files = sorted(set([file[:14] for file in xFiles]))

# Create empty dataframes for x and y data
xData = pd.DataFrame()
data_list = []

for file in files:
    y_time = pd.read_csv(file + '__y_time.csv', names=['time'])
    x_data = pd.read_csv(file + '__x.csv', names = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z'])
    x_time = pd.read_csv(file + '__x_time.csv', names=['time'])

    x_data['time'] = x_time['time']
    x_data = x_data.merge(y_time, how='outer', on='time')
    x_data.sort_values('time', inplace=True)

    x_data = x_data.interpolate()

    new_data_list = []

    i = .2
    for datapoint in x_data.values:
      if math.isclose(datapoint[6], i/10):
        new_data_list.append(datapoint)
        i += 1
    
    new_data = pd.DataFrame(new_data_list, columns=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'time'])
    
    data = new_data.drop('time', axis=1)

    # Transform the data using the fitted scaler
    x_data = scaler.transform(data)
    o_len = len(x_data)
    while len(x_data) % 10 != 0:
      last_row = x_data[-1]
      x_data = np.vstack([x_data, last_row])
    x_data = pd.DataFrame(x_data)

    # Reshape the data
    window=10
    window_x=[]
    for i in range(0, len(x_data), window):
        window_x.append([x_data.values[i:i+window]])
    window_x=np.array(window_x).reshape(-1,window,6)

    # Make predictions and reshape the output
    y_pred = model.predict(window_x)
    y_hat = []
    for i in y_pred:
        for j in i:
            y_hat.append(np.argmax(j))

    while len(y_hat) != o_len:
      y_hat = np.delete(y_hat, -1)

    # Save predictions to a file
    finaldf = pd.DataFrame({'y_pred': y_hat})
    test_path = "/content/Predictions/"
    final_path = test_path + file + "__y.csv"
    finaldf.to_csv(final_path, index=False, header=False)

final_pred_09 = pd.read_csv('/content/Predictions/subject_009_01__y.csv')
final_pred_10 = pd.read_csv('/content/Predictions/subject_010_01__y.csv')
final_pred_11 = pd.read_csv('/content/Predictions/subject_011_01__y.csv')
final_pred_12 = pd.read_csv('/content/Predictions/subject_012_01__y.csv')

print(final_pred_09.value_counts())
print(final_pred_10.value_counts())
print(final_pred_11.value_counts())
print(final_pred_12.value_counts())

30/30 [==============================] - 1s 11ms/step
9473
0
0    7128
1    1016
2     934
3     394
dtype: int64
